In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import db_password
import psycopg2
import pandas as pd


In [2]:
engine = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    password = db_password,
    host = "dataviz-db.csxrf9ti2aba.us-east-2.rds.amazonaws.com",
    port = 5432)
cursor = engine.cursor()

In [3]:
#use Pandas to see all data in table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=engine)

,table_name,column_name,data_type,table_schema
0,bigquery_hits_products,productcouponcode,character varying,public
1,bigquery_hits_products,browser,character varying,public
2,bigquery_hits_products,productquantity,integer,public
3,bigquery_hits_products,hour,integer,public
4,bigquery_hits_products,step,integer,public
...,...,...,...,...
104,vine_table,total_votes,integer,public
105,vine_table,vine,text,public
106,vine_table,star_rating,integer,public
107,vine_table,helpful_votes,integer,public


In [4]:
#Pull all info into pandas dataframe from RDS dB
sql = """
SELECT *
from bigquery_totals2
"""
bigquery_totals_df = pd.read_sql(sql,con = engine)
#plug NaN's as 0's
bigquery_totals_df = bigquery_totals_df.fillna(0)
bigquery_totals_df.head(n = 10)


,fullvisitorid,visitid,visitnumber,visitstarttime,date,socialengagementtype,channelgrouping,browser,operatingsystem,devicecategory,...,istruedirect,bounces,hits,newvisits,pageviews,sessionqualitydim,totaltransactionrevenue,transactionrevenue,transactions,visits
0,0000062267706107999,1499645960,1,1499645960,2017-07-09,Not Socially Engaged,Affiliates,Chrome,Windows,desktop,...,0,1.0,1,1.0,1.0,1.0,0.0,0.0,0.0,1
1,0000085059828173212,1500505105,1,1500505105,2017-07-19,Not Socially Engaged,Direct,Safari,iOS,mobile,...,True,1.0,1,1.0,1.0,1.0,0.0,0.0,0.0,1
2,0000197671390269035,1493772870,1,1493772870,2017-05-02,Not Socially Engaged,Social,Android Webview,Android,mobile,...,0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,1
3,000026722803385797,1496658394,1,1496658394,2017-06-05,Not Socially Engaged,Organic Search,Safari,Macintosh,desktop,...,0,0.0,3,1.0,2.0,0.0,0.0,0.0,0.0,1
4,0000436683523507380,1500504900,1,1500504900,2017-07-19,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,...,0,0.0,2,1.0,2.0,1.0,0.0,0.0,0.0,1
5,0000436683523507380,1500989127,2,1500989127,2017-07-25,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,...,True,0.0,8,0.0,8.0,6.0,0.0,0.0,0.0,1
6,0000450371054833295,1500303787,1,1500303787,2017-07-17,Not Socially Engaged,Referral,Chrome,Linux,desktop,...,0,1.0,1,1.0,1.0,1.0,0.0,0.0,0.0,1
7,0000458812883559498,1495066298,1,1495066298,2017-05-17,Not Socially Engaged,Organic Search,Chrome,Linux,desktop,...,0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,1
8,0000572434142265465,1494990076,1,1494990076,2017-05-16,Not Socially Engaged,Referral,Chrome,Chrome OS,desktop,...,0,0.0,2,1.0,2.0,0.0,0.0,0.0,0.0,1
9,0000572434142265465,1500605115,2,1500605115,2017-07-20,Not Socially Engaged,Referral,Chrome,Chrome OS,desktop,...,True,0.0,3,0.0,3.0,1.0,0.0,0.0,0.0,1


In [5]:
#Action_type is the column we're going to want to predict.
#This column dictates what kind of action the visitor took on an item

#Drop in Machine Learning logistic regression libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [6]:
bigquery_totals_df.dtypes

fullvisitorid               object
visitid                      int64
visitnumber                  int64
visitstarttime               int64
date                        object
socialengagementtype        object
channelgrouping             object
browser                     object
operatingsystem             object
devicecategory              object
continent                   object
subcontinent                object
country                     object
region                      object
metro                       object
city                        object
referralpath                object
source                      object
medium                      object
istruedirect                object
bounces                    float64
hits                         int64
newvisits                  float64
pageviews                  float64
sessionqualitydim          float64
totaltransactionrevenue    float64
transactionrevenue         float64
transactions               float64
visits              

In [7]:
bigquery_totals_df.nunique()

fullvisitorid              12446
visitid                    15952
visitnumber                  152
visitstarttime             15973
date                          93
socialengagementtype           1
channelgrouping                7
browser                       19
operatingsystem               12
devicecategory                 3
continent                      6
subcontinent                  23
country                      141
region                       145
metro                         46
city                         209
referralpath                 242
source                        68
medium                         6
istruedirect                   2
bounces                        2
hits                          97
newvisits                      2
pageviews                     73
sessionqualitydim             96
totaltransactionrevenue      221
transactionrevenue           202
transactions                   4
visits                         1
dtype: int64

In [8]:
# # Lets work on reducing value dimensions in some of our columns to do oneHotEncoding
# print(bigquery_totals_df.browser.value_counts())
# # We can reduce by counting all browser values under 100 as OTHER

# print(bigquery_totals_df.subcontinent.value_counts())
# # We can reduce by count all subcontinent values under 500 as OTHER

# print(bigquery_totals_df.country.value_counts())
# bigquery_totals_df.country.value_counts().plot.density()
# bigquery_totals_df.country.value_counts()[bigquery_totals_df.country.value_counts()>250]
# # We can reduce by count of all countries values under 250 as OTHER

# print(bigquery_totals_df.region.value_counts())
# bigquery_totals_df.region.value_counts()[bigquery_totals_df.region.value_counts()>100]
# # A lot of "not available in demo dataset". I think we should remove this column.

# bigquery_totals_df.metro.value_counts()
# # A lot of "not available in demo dataset". I think we should remove this column.

# bigquery_totals_df.city.value_counts()
# # A lot of "not available in demo dataset". I think we should remove this column.

# bigquery_totals_df.referralpath.value_counts()
# bigquery_totals_df.referralpath.value_counts()[bigquery_totals_df.referralpath.value_counts()>10]
# # Unsure how well this column will be in our model. I think we should remove this column 

# bigquery_totals_df.source.value_counts()
# bigquery_totals_df.source.value_counts()[bigquery_totals_df.source.value_counts()>50]
# # We can reduce by count of all sources under 50 as OTHER





In [9]:
#REPLACEEEEEEEEEEEEEEE
replace_browser_types = list(bigquery_totals_df.browser.value_counts()[bigquery_totals_df.browser.value_counts()<100].index)
replace_subcontinent_types = list(bigquery_totals_df.subcontinent.value_counts()[bigquery_totals_df.subcontinent.value_counts()<500].index)
replace_country_types = list(bigquery_totals_df.country.value_counts()[bigquery_totals_df.country.value_counts()<250].index)
replace_source_types = list(bigquery_totals_df.source.value_counts()[bigquery_totals_df.source.value_counts()<50].index)



In [10]:
#Replace in dataframe
for browser_replace in replace_browser_types:
    bigquery_totals_df.browser = bigquery_totals_df.browser.replace(browser_replace,"Other")

for subcontinent_replace in replace_subcontinent_types:
    bigquery_totals_df.subcontinent = bigquery_totals_df.subcontinent.replace(subcontinent_replace,"Other")
    
for country_replace in replace_country_types:
    bigquery_totals_df.country = bigquery_totals_df.country.replace(country_replace,"Other")
    
for source_replace in replace_source_types:
    bigquery_totals_df.source = bigquery_totals_df.source.replace(source_replace,"Other")
    
    
    
bigquery_totals_df.nunique() 

fullvisitorid              12446
visitid                    15952
visitnumber                  152
visitstarttime             15973
date                          93
socialengagementtype           1
channelgrouping                7
browser                        9
operatingsystem               12
devicecategory                 3
continent                      6
subcontinent                  10
country                       11
region                       145
metro                         46
city                         209
referralpath                 242
source                        10
medium                         6
istruedirect                   2
bounces                        2
hits                          97
newvisits                      2
pageviews                     73
sessionqualitydim             96
totaltransactionrevenue      221
transactionrevenue           202
transactions                   4
visits                         1
dtype: int64

In [11]:
#Create a list of the categorical variables
bigquery_totals_cat = bigquery_totals_df.dtypes[bigquery_totals_df.dtypes == "object"].index.tolist()


# From the list of categorical variables we DONT want some of them. We'll .remove() the columns we want from the list
removal_list = ['fullvisitorid','region','metro','city','referralpath','istruedirect']

for elem in removal_list:
    bigquery_totals_cat.remove(elem)
# This is pretty ugly. 

# Fix Boolean istruedirect column from True/None to 1/0
# bigquery_totals_df["istruedirect"] = bigquery_totals_df["istruedirect"].astype(int)

# #get number of unique values per category column
bigquery_totals_df[bigquery_totals_cat].head()

,date,socialengagementtype,channelgrouping,browser,operatingsystem,devicecategory,continent,subcontinent,country,source,medium
0,2017-07-09,Not Socially Engaged,Affiliates,Chrome,Windows,desktop,Americas,South America,Other,Partners,affiliate
1,2017-07-19,Not Socially Engaged,Direct,Safari,iOS,mobile,Americas,Northern America,United States,(direct),(none)
2,2017-05-02,Not Socially Engaged,Social,Android Webview,Android,mobile,Americas,Northern America,United States,m.facebook.com,referral
3,2017-06-05,Not Socially Engaged,Organic Search,Safari,Macintosh,desktop,Europe,Northern Europe,United Kingdom,google,organic
4,2017-07-19,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,Americas,Northern America,United States,(direct),(none)


In [12]:
#slap a OneHotEncoding on this for categorical columns
enc = OneHotEncoder(sparse=False)

#fit and transform the encoding using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(bigquery_totals_df[bigquery_totals_cat]))

#add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(bigquery_totals_cat)
encode_df.head()

,date_2017-05-01,date_2017-05-02,date_2017-05-03,date_2017-05-04,date_2017-05-05,date_2017-05-06,date_2017-05-07,date_2017-05-08,date_2017-05-09,date_2017-05-10,...,source_google,source_google.com,source_m.facebook.com,source_youtube.com,medium_(none),medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#merge one-hot encoded features and drop the originals
bigquery_totals_df1 = bigquery_totals_df.merge(encode_df,left_index = True, right_index = True)
bigquery_totals_df1 = bigquery_totals_df1.drop(columns = bigquery_totals_cat+removal_list)
#bigquery_hits_df1 = bigquery_hits_df1.drop(columns = remove_these_columns)
bigquery_totals_df1.head()
#bigquery_hits_df1.productprice

,visitid,visitnumber,visitstarttime,bounces,hits,newvisits,pageviews,sessionqualitydim,totaltransactionrevenue,transactionrevenue,...,source_google,source_google.com,source_m.facebook.com,source_youtube.com,medium_(none),medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral
0,1499645960,1,1499645960,1.0,1,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1500505105,1,1500505105,1.0,1,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1493772870,1,1493772870,1.0,1,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1496658394,1,1496658394,0.0,3,1.0,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1500504900,1,1500504900,0.0,2,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


SyntaxError: invalid syntax (<ipython-input-14-fe8450893fce>, line 1)

In [21]:
#Remove Action type from our features dataset
y = bigquery_totals_df1.transactions

X = bigquery_totals_df1.drop(columns =['transactions'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [17]:
#Standar scaler time lets get it

#create a StandardScaler instance
scaler = StandardScaler()

#Fit the StandardScaler
X_scaler = scaler.fit(X_train)



In [20]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=1000)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 1.000
